In [42]:
#!pip install miditok
#!pip install symusic
#!pip install glob
#!pip install torch

In [43]:
import glob
import random
from typing import List
from collections import defaultdict

import os
import pandas as pd

import numpy as np
from numpy.random import choice

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from symusic import Score
from miditok import REMI, TokenizerConfig, TokenizerConfig
from miditok.pytorch_data import DatasetMIDI, DataCollator

from midi2audio import FluidSynth # Import library
from IPython.display import Audio, display

from pretty_midi import PrettyMIDI

In [44]:
# Uses 'cuda' if a gpu is detected. Otherwise uses cpu
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Can also set manually
#DEVICE = 'cpu'
#DEVICE = 'cuda'

print(DEVICE)

cuda


In [45]:
ROOT = "maestro-v3.0.0"            # change if you unpacked elsewhere
meta = pd.read_csv(os.path.join(ROOT, "maestro-v3.0.0.csv"))

def list_midi_files(split):
    paths = meta.loc[meta["split"] == split, "midi_filename"]
    return [os.path.join(ROOT, p) for p in paths]

train_files = list_midi_files("train")        # 962 MIDI files
val_files   = list_midi_files("validation")   # 137
test_files  = list_midi_files("test")         # 177


In [46]:
type(train_files[0])
train_files[0].encode('utf-8').decode('utf-8')
print(train_files[0].encode('utf-8'))
str.encode(train_files[0], 'utf-8')

b'maestro-v3.0.0\\2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi'


b'maestro-v3.0.0\\2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi'

Tokenizer

In [47]:
# Define tokenizer parameters
TOKENIZER_PARAMS = {
    "use_chords": True,
    "use_tempos": True,
    "use_time_signatures": True,
    "use_key_signatures": True,
}

# Create the tokenizer configuration
config = TokenizerConfig(**TOKENIZER_PARAMS)

# Initialize the REMI tokenizer with the configuration
tokenizer = REMI(config)


train_dataset = DatasetMIDI(
    files_paths=train_files,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
test_dataset = DatasetMIDI(
    files_paths=test_files,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
collator = DataCollator(tokenizer.pad_token_id)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collator)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collator)

In [48]:
len(train_loader), len(test_loader)

(241, 45)

### Transformer

In [49]:
class MusicTransformer(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, num_heads=8, num_layers=6, dropout=0.1, max_seq_len=1024):
        super(MusicTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_encoder = nn.Parameter(self._generate_positional_encoding(max_seq_len, embedding_dim), requires_grad=False)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=embedding_dim * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc_out = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        # x: (batch_size, seq_len)
        x = self.embedding(x) + self.pos_encoder[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        return self.fc_out(x)

    def _generate_positional_encoding(self, max_len, d_model):
        """Creates sinusoidal positional encoding matrix"""
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)  # shape: (1, max_len, d_model)


Training

In [50]:
def train(model, train_loader, val_loader, vocab_size, num_epochs=20, lr=0.001, device=DEVICE):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        # --------- Training ---------
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            batch = batch['input_ids'].to(device)  # (batch_size, seq_length)

            inputs = batch[:, :-1]
            targets = batch[:, 1:]

            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.reshape(-1, vocab_size)
            targets = targets.reshape(-1)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # --------- Validation ---------
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch['input_ids'].to(device)

                inputs = batch[:, :-1]
                targets = batch[:, 1:]

                outputs = model(inputs)
                outputs = outputs.reshape(-1, vocab_size)
                targets = targets.reshape(-1)

                loss = criterion(outputs, targets)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

In [51]:
vocab_size = tokenizer.vocab_size
embedding_dim = 256
hidden_dim = 512
num_layers = 2

model = MusicTransformer(vocab_size, embedding_dim=256, num_heads=8, num_layers=6)
train(model, train_loader, test_loader, vocab_size)

Epoch 1/20 | Train Loss: 4.2482 | Val Loss: 4.5187
Epoch 2/20 | Train Loss: 4.4970 | Val Loss: 4.4702
Epoch 3/20 | Train Loss: 4.5175 | Val Loss: 4.4673
Epoch 4/20 | Train Loss: 4.5121 | Val Loss: 4.4685
Epoch 5/20 | Train Loss: 4.5075 | Val Loss: 4.4656
Epoch 6/20 | Train Loss: 4.5057 | Val Loss: 4.4665
Epoch 7/20 | Train Loss: 4.5025 | Val Loss: 4.4625
Epoch 8/20 | Train Loss: 4.5037 | Val Loss: 4.4613
Epoch 9/20 | Train Loss: 4.5000 | Val Loss: 4.4619
Epoch 10/20 | Train Loss: 4.4997 | Val Loss: 4.4632
Epoch 11/20 | Train Loss: 4.5014 | Val Loss: 4.4630
Epoch 12/20 | Train Loss: 4.4989 | Val Loss: 4.4679
Epoch 13/20 | Train Loss: 4.5010 | Val Loss: 4.4582
Epoch 14/20 | Train Loss: 4.4987 | Val Loss: 4.4662
Epoch 15/20 | Train Loss: 4.4992 | Val Loss: 4.4617
Epoch 16/20 | Train Loss: 4.5005 | Val Loss: 4.4605
Epoch 17/20 | Train Loss: 4.5000 | Val Loss: 4.4602
Epoch 18/20 | Train Loss: 4.4998 | Val Loss: 4.4648
Epoch 19/20 | Train Loss: 4.4988 | Val Loss: 4.4602
Epoch 20/20 | Train L

Sampling

In [57]:
def sample(model, start_token, tokenizer, max_length=512, temperature=1.0, device=DEVICE):
    model.eval()

    # Build ID → string mapping
    if hasattr(tokenizer, 'vocab') and isinstance(tokenizer.vocab, dict):
        id_to_token = {v: k for k, v in tokenizer.vocab.items()}
    elif hasattr(tokenizer, '_vocab'):
        id_to_token = {i: tok for i, tok in enumerate(tokenizer._vocab)}
    else:
        raise RuntimeError("Tokenizer vocab not found")

    generated = [start_token]
    input_seq = torch.tensor([generated], dtype=torch.long, device=device)

    while len(generated) < max_length:
        with torch.no_grad():
            logits = model(input_seq)
            next_logits = logits[0, -1] / temperature
            probs = F.softmax(next_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()

        token_str = id_to_token.get(next_token, "")

        # Always add bar/position/timeshift
        if token_str.startswith(("Bar", "TimeShift", "Position")):
            generated.append(next_token)

        # If it's a pitch, follow with velocity and duration
        elif token_str.startswith("Pitch_"):
            generated.append(next_token)

            # Sample a Velocity token
            velocity_ids = [i for i, tok in id_to_token.items() if tok.startswith("Velocity_")]
            generated.append(random.choice(velocity_ids))

            # Sample a Duration token
            duration_ids = [i for i, tok in id_to_token.items() if tok.startswith("Duration_")]
            generated.append(random.choice(duration_ids))

        # Stop on EOS or PAD
        if token_str in ("EOS_None"):
            break

        input_seq = torch.tensor([generated], dtype=torch.long, device=device)

    return generated


In [61]:
start_token = tokenizer.special_tokens_ids[1]
generated_sequence = sample(model, start_token, tokenizer, max_length=1024)
#generated_sequence2 = sample(model, generated_sequence1[-1], tokenizer, max_length=1024)

#generated_sequence = generated_sequence1 + generated_sequence2

print("Generated token sequence:")
print(generated_sequence)
print("num tokens:", len(generated_sequence))

Generated token sequence:
[1, 44, 94, 171, 202, 197, 217, 57, 102, 128, 54, 99, 131, 44, 108, 146, 4, 40, 105, 167, 52, 94, 148, 201, 4, 53, 116, 159, 56, 115, 170, 64, 110, 161, 200, 24, 122, 158, 44, 93, 159, 27, 118, 172, 53, 108, 184, 57, 112, 164, 191, 35, 111, 181, 4, 14, 95, 186, 207, 48, 105, 164, 218, 45, 94, 168, 66, 123, 171, 24, 100, 160, 68, 113, 175, 190, 198, 50, 111, 171, 207, 208, 41, 121, 176, 59, 102, 155, 216, 198, 35, 109, 128, 31, 107, 172, 21, 104, 159, 213, 52, 119, 156, 39, 107, 165, 40, 112, 186, 58, 106, 156, 194, 57, 98, 125, 206, 16, 118, 183, 41, 116, 166, 190, 202, 50, 112, 145, 199, 58, 116, 151, 216, 64, 108, 178, 15, 98, 185, 53, 93, 134, 215, 210, 31, 119, 154, 58, 102, 129, 215, 23, 122, 133, 45, 118, 163, 46, 94, 156, 191, 57, 111, 125, 192, 28, 119, 166, 25, 113, 141, 193, 190, 43, 119, 145, 205, 4, 196, 42, 116, 159, 219, 33, 123, 173, 39, 117, 133, 216, 217, 4, 63, 107, 161, 52, 100, 146, 45, 104, 152, 57, 93, 126, 42, 95, 167, 218, 4, 196, 37, 1

In [62]:
output_score = tokenizer.decode([generated_sequence])
output_score.dump_midi(f"transformer.mid")

In [63]:
pretty_midi = PrettyMIDI("transformer.mid")
print("Duration (seconds):", pretty_midi.get_end_time())
for i, instrument in enumerate(pretty_midi.instruments):
    print(f"{instrument.name or 'Unnamed'}:", len(instrument.notes), "notes")

Duration (seconds): 48.10782449999999
Acoustic Grand Piano: 279 notes


In [56]:
shortest_time = 1000
for i in range(100):
    start_token = tokenizer.special_tokens_ids[1]
    generated_sequence = sample(model, start_token, tokenizer, max_length=1024)

    output_score = tokenizer.decode([generated_sequence])
    output_score.dump_midi(f"transformer_temp.mid")

    pretty_midi_temp = PrettyMIDI("transformer_temp.mid")
    if(pretty_midi_temp.get_end_time() < shortest_time):
        output_score = tokenizer.decode([generated_sequence])
        output_score.dump_midi(f"transformer.mid")
        shortest_time = pretty_midi.get_end_time()

pretty_midi = PrettyMIDI("transformer.mid")
print("Duration (seconds):", pretty_midi.get_end_time())
for i, instrument in enumerate(pretty_midi.instruments):
    print(f"{instrument.name or 'Unnamed'}:", len(instrument.notes), "notes")


Duration (seconds): 48.10782449999999
Acoustic Grand Piano: 267 notes
